In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from config import census_key


c = Census(census_key, year=2013)
############

file = 'zipcodes_stl2.csv'
df = pd.read_csv(file)
df_small = df.iloc[0:5,:]
df_small
############


df_census = pd.DataFrame()

for zip in df_small['ZIP Code']:
    census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                                  "B19301_001E",
                                  "B17001_002E"), {'for': 'zip code tabulation area:'+str(zip)})
#     print(census_data)
    df_new = pd.DataFrame(census_data)
    df_census = df_census.append(df_new)
# df_census.head()
# Column Reordering
df_census = df_census.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "ZIP Code"})

# Add in Poverty Rate (Poverty Count / Population)
df_census["Poverty Rate"] = 100 * \
    df_census["Poverty Count"].astype(
        int) / df_census["Population"].astype(int)

# Final DataFrame
df_census = df_census[["ZIP Code", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count","Poverty Rate"]]

df_census['ZIP Code'] = pd.to_numeric(df_census['ZIP Code'])
df_census.head()
#######

df_merge = pd.merge(df_census, df_small, on=('ZIP Code'))
df_merge
######

#New Dependencies

import json
import pprint
# import requests
# import sys
import urllib
from urllib.parse import quote
    
from config import yelp_key
########

API_KEY=yelp_key

API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
MATCH_PATH = '/v3/businesses/matches'
SEARCH_PHONE_PATH = '/v3/businesses/search/phone'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.

# Defaults for our simple example.
DEFAULT_TERM = 'dinner'
DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 10
#########


def request(host, path, api_key, url_params=None):
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))
#     print(headers)

    response = requests.request('GET', url, headers=headers, params=url_params)
#     my_url=url
    return response.json() #, my_url

#######


df_merge.dtypes
df_merge['Latitude'] = df_merge['Latitude'].astype(str)
df_merge['Longitude'] = df_merge['Longitude'].astype(str)
df_merge.dtypes
########

df_businesses = pd.DataFrame()

df_merge['Latitude'] = df_merge['Latitude'].astype(str)
df_merge['Longitude'] = df_merge['Longitude'].astype(str)

for row in df_merge:
    row_lat = row['Latitude']
    row_long = row['Longitude']
    print(row_lat,row_long)
    url_params={'latitude':row_lat,'longitude':row_long, 'radius':'10000'}
#     url_params={'latitude':'38.739650','longitude':'-90.32178', 'radius':'10000'}
    dict_businesses = request(API_HOST, SEARCH_PATH, API_KEY, url_params)

    df_businesses['business_id']=[business["id"] for business in dict_businesses['businesses']]
    df_businesses['name']=[business["name"]for business in dict_businesses['businesses']]
    df_businesses['rating']=[business["rating"]for business in dict_businesses['businesses']]
    df_businesses['zip_code']=[business['location']["zip_code"]for business in dict_businesses['businesses']]
    df_businesses['review_count']=[business["review_count"]for business in dict_businesses['businesses']]
    # try:
    #     df_businesses['price']= [business["price"].map('\$'.format)for business in dict_businesses['businesses']]
    # except:
    #     df_businesses['price']=0
    try:
        if dict_businesses['businesses']['price'] == "$":
            df_businesses['price'] = 1
        elif dict_businesses['business']['price'] == '$$':
            df_businesses['price'] = 2
    except:
        df_businesses['price'] = 0

df_businesses
# pprint.pprint(dict_businesses)